<a href="https://colab.research.google.com/github/lbhagavan/stanford_LLM_Leela/blob/homework/LeelaMainProjectwithUI_TECH16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project to read CA fire websites and train the models  Using RAG
#Better Q&A with AI crew




# Standard imports

In [6]:
%pip install openai
!pip install llama-index --upgrade
!pip install llama-index-readers-web


from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

from llama_index.core import VectorStoreIndex
from llama_index.readers.web import WholeSiteReader
from llama_index.core import Document

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [7]:
def scrape_data(url, prefix, max_depth=6, current_depth=0, visited=None):
    # st.write(Document.__init__.__annotations__)
    if visited is None:
        visited = set()  # Initialize the set of visited URLs

    if current_depth > max_depth:
        return []

    if url in visited:
        return []  # Avoid revisiting the same URL

    visited.add(url)
    # st.write(url)
    documents = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract text data from all paragraphs
        page_content = [p.get_text().strip() for p in soup.find_all('p') if p.get_text().strip()]

        # Convert each piece of text data to a Document object
        for text in page_content:
            doc = Document(text=text, metadata={"url": url, "depth": current_depth})
            documents.append(doc)

        # Find all links on the page to follow
        for link in soup.find_all('a', href=True):
            full_url = urljoin(url, link['href'])
            if full_url.startswith(prefix):
                documents.extend(scrape_data(full_url, prefix, max_depth, current_depth + 1, visited))

    except requests.RequestException as e:
        st.write(f"An error occurred while scraping {url}: {e}")

    return documents

# Set the base URL and prefix for the scraper
base_url = "https://www.fire.ca.gov/"
prefix = "https://www.fire.ca.gov/"

# Scrape data up to a depth of 6
documents = scrape_data(base_url, prefix, max_depth=6)
index = VectorStoreIndex.from_documents(documents)



In [9]:
query_engine = index.as_query_engine()

In [10]:
response = query_engine.query("can you please summrize 2022 cal fires")
print(response)


In 2022, the total acreage burned in California due to wildfires was less than 400,000 acres, which was significantly lower than the 5-year average of over 2,300,000 acres.


In [11]:
response = query_engine.query("between 2022 or 2023 cal fires, which season is good") # used the word " be specific" to get clear message
print(response)



The 2023 California fire season is considered better than the 2022 season.


In [12]:
response = query_engine.query("can you please summerize borel fire")
print(response)

The Borel Fire has affected an area of 59,288 acres and is currently 94% contained. There are 144 total personnel involved in managing the fire.
